# Weather Forecast - Modelling

## Import Data for Modelling

Import Libraries

In [1]:
import numpy as np
import pandas as pd

Importing Data

In [2]:
filename = 'modelling_data.csv'
data_df = pd.read_csv(filename)
data_df.head()

,Date,Atmospheric Pressure,Minimum Temperature,Maximum Temperature,Relative Humidity,Wind Speed
0,2018-01-01,1019.5,16.3,26.1,25.333333,9.666667
1,2018-01-02,1019.6,16.1,27.4,27.583333,12.000000
2,2018-01-03,1019.9,15.7,27.2,24.416667,11.666667
3,2018-01-04,1021.4,16.0,29.0,27.166667,12.333333
4,2018-01-05,1021.2,14.2,26.9,29.916667,12.333333


Computing the Mean Temperature

In [3]:
data_df['Mean Temperature'] = pd.Series(np.mean([data_df['Minimum Temperature'], data_df['Maximum Temperature']], axis=0))
data_df.head()

,Date,Atmospheric Pressure,Minimum Temperature,Maximum Temperature,Relative Humidity,Wind Speed,Mean Temperature
0,2018-01-01,1019.5,16.3,26.1,25.333333,9.666667,21.20
1,2018-01-02,1019.6,16.1,27.4,27.583333,12.000000,21.75
2,2018-01-03,1019.9,15.7,27.2,24.416667,11.666667,21.45
3,2018-01-04,1021.4,16.0,29.0,27.166667,12.333333,22.50
4,2018-01-05,1021.2,14.2,26.9,29.916667,12.333333,20.55


In [4]:
data_df.shape

(1096, 7)

## LSTM - Weather Forecasting

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Dropout, BatchNormalization, Bidirectional, Conv1D

from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
from sklearn.metrics import mean_squared_error

from keras.layers import ConvLSTM2D

Using TensorFlow backend.


Convert DataFrame to Numpy Array

In [ ]:
date_series = data_df['Date']
data_array = data_df.iloc[:, 1:].values
data_array.shape, data_array.dtype, date_series.shape

Scaling of Dataset to avoid gradient explosion. 

__N.B:__ One importance of fit-transform is that the scaling can easily be inverted.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(data_array)
data_array = scaler.transform(data_array)
data_array.shape, data_array.dtype

Preparing the training and testing data for LSTM

In [ ]:
def train_test_data_for_lstm(data, n_past, n_future=1, view_shape=False):
    """
    Function to determine the training and testing data for a recurrent neural network modelling.
    
    Input:
    data - numpy array of the data to be modelled
    n-past - the number of days in the past to be utilised for modelling
    n-future - the date in the future to be predicted.
    
    Output:
    trainX - the training features
    trainY - the training targets
    """
    
    # Define list to store training and testing datasets
    X = []
    Y = []
    
    for i in range(n_past, len(data) - n_future +1):
        X.append(data[i - n_past:i, 0:data.shape[1] - 1])
        Y.append(data[i + n_future - 1:i + n_future, data.shape[1] - 1])

    X, Y = np.array(X), np.array(Y)
    
    if view_shape:
        print('trainX shape == {}.'.format(X.shape))
        print('trainY shape == {}.'.format(Y.shape))
        
    return X, Y   

In [ ]:
trainX, trainY = train_test_data_for_lstm(data_array, 28, 1, True)

In [ ]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
weather_model = model.fit(trainX, trainY, epochs=5, batch_size=16, validation_split=0.1, verbose=1)

Plot the graph of the training loss and the validation loss for the developed model.

In [ ]:
import seaborn as sb